# Demo GPU computation with CUDA
* `iqwaveform` and `edge_analyzer` have been developed to accept `numpy` (CPU) or `cupy` (CUDA GPU) arrays interchangeably
* This shows the evaluation on a Jetson TX2
* Beginning from the `Channel power time-series` section, the code cells are identical to the CPU (numpy) compute notebook

### Generate fake data

In [ ]:
from channel_analysis.io import simulated_awgn
import cupy as cp
import iqwaveform.cuda

# this takes a while the first time, so get it out of the way now
iqwaveform.cuda.build()

### Analysis bandpass filter

In [ ]:
from channel_analysis import waveform
from iqwaveform import fourier
from cupyx.profiler import benchmark
from channel_analysis.io import simulated_awgn
import cupy as cp

import importlib
fourier = importlib.reload(fourier)

duration = 1
sample_rate_Hz = 15.36e6  # 11.52e6
analysis_bandwidth_Hz = 10e6

iq = simulated_awgn(
    duration, sample_rate_Hz, power=sample_rate_Hz / analysis_bandwidth_Hz, xp=cp, pinned_cuda=True
)
iq = fourier.ola_filter(
    iq,
    fs=sample_rate_Hz,
    fft_size=1024,
    window='hamming',
    passband=(-analysis_bandwidth_Hz / 2, analysis_bandwidth_Hz / 2),
    out='shared'
)

cp.cuda.runtime.deviceSynchronize()

In [ ]:
# def profile():
#     ret = fourier.ola_filter(iq, fs=sample_rate_Hz, fft_size=1024, window='hamming', passband=(-analysis_bandwidth_Hz/2, analysis_bandwidth_Hz/2)).get()

# benchmark(profile, n_repeat=10)

### Channel power time-series

In [ ]:
pvt = waveform.power_time_series(
    iq,
    sample_rate_Hz=sample_rate_Hz,
    analysis_bandwidth_Hz=analysis_bandwidth_Hz,
    detector_period=10e-3,
    detectors=('rms', 'peak'),
).to_xarray()

In [ ]:
def profile():
    return waveform.power_time_series(
        iq,
        sample_rate_Hz=sample_rate_Hz,
        analysis_bandwidth_Hz=analysis_bandwidth_Hz,
        detector_period=10e-3,
        detectors=('rms', 'peak'),
    )

# benchmark(profile, n_repeat=10)
benchmark(lambda: profile().to_xarray(), n_repeat=10)

In [ ]:
from iqwaveform import powtodB
from matplotlib import pyplot as plt
from channel_analysis import figures

plt.style.use('iqwaveform.ieee_double_column')

fig, ax = plt.subplots()

pvt.plot.line(x='time_elapsed', ax=ax)

figures.label_axis(ax.xaxis, pvt, 'time_elapsed')
figures.label_axis(ax.yaxis, pvt, tick_units=False)
figures.label_legend(ax, pvt, 'power_detector')

### Cyclic channel power

In [ ]:
cyclic_power = waveform.cyclic_channel_power(
    iq,
    sample_rate_Hz,
    analysis_bandwidth_Hz=analysis_bandwidth_Hz,
    cyclic_period=10e-3,
    detector_period=1e-3 / 15 / 4,
    detectors=('rms', 'peak'),
    cyclic_statistics=('min', 'mean', 'max'),
).to_xarray()

In [ ]:
def profile():
    return waveform.cyclic_channel_power(
        iq,
        sample_rate_Hz,
        analysis_bandwidth_Hz=analysis_bandwidth_Hz,
        cyclic_period=10e-3,
        detector_period=1e-3 / 15 / 4,
        detectors=pvt.power_detector.values,
        cyclic_statistics=('min', 'mean', 'max'),
    )

# benchmark(profile, n_repeat=10)
benchmark(lambda: profile().to_xarray(), n_repeat=10)

In [ ]:
fig, ax = plt.subplots()

time = cyclic_power.cyclic_lag

for i, detector in enumerate(cyclic_power.power_detector.values):
    a = cyclic_power.sel(power_detector=detector)

    ax.plot(time, powtodB(a.sel(cyclic_statistic='mean')), color=f'C{i}')

for i, detector in enumerate(cyclic_power.power_detector.values):
    a = cyclic_power.sel(power_detector=detector)

    ax.fill_between(
        time,
        powtodB(a.sel(cyclic_statistic='min')),
        powtodB(a.sel(cyclic_statistic='max')),
        color=f'C{i}',
        alpha=0.25,
        lw=0,
        rasterized=True,
    )

figures.label_axis(ax.xaxis, cyclic_power, 'cyclic_lag')
figures.label_axis(ax.yaxis, cyclic_power, tick_units=False)
figures.label_legend(ax, cyclic_power, 'power_detector')

### Persistence Spectrum

In [ ]:
from importlib import reload

waveform = reload(waveform)
waveform.fourier = reload(waveform.fourier)

frequency_resolution = 15e3
spectrum_quantiles = [0.5, 0.75, 0.95, 0.99, 1]

spectrum = waveform.persistence_spectrum(
    iq,
    sample_rate_Hz=sample_rate_Hz,
    analysis_bandwidth_Hz=analysis_bandwidth_Hz,
    window=('dpss', 5),
    resolution=frequency_resolution,
    quantiles=spectrum_quantiles,
    fractional_overlap=0.0,
    truncate=True,
    dB=True,
).to_xarray()

In [ ]:
fig, ax = plt.subplots()

spectrum.plot.line(x='baseband_frequency', ax=ax)
figures.label_axis(ax.xaxis, spectrum, 'baseband_frequency')
figures.label_axis(ax.yaxis, spectrum, tick_units=False)
figures.label_legend(ax, spectrum, 'persistence_statistic')

In [ ]:
def profile():
    return waveform.persistence_spectrum(
        iq,
        sample_rate_Hz=sample_rate_Hz,
        analysis_bandwidth_Hz=analysis_bandwidth_Hz,
        window=('dpss', 3),
        resolution=frequency_resolution,
        quantiles=spectrum_quantiles,
        fractional_overlap=0.0,
        truncate=True,
        dB=True,
    )

benchmark(profile, n_repeat=1)
# benchmark(lambda: profile().to_xarray(), n_repeat=10)

# %%timeit -n1 -r1

### Amplitude Probability Distribution

In [ ]:
apd_bins = dict(power_low=-60.0, power_high=15.0, power_count=55 * 4 + 1)

apd = waveform.amplitude_probability_distribution(
    iq, analysis_bandwidth_Hz=analysis_bandwidth_Hz, **apd_bins
).to_xarray()

In [ ]:
%%timeit -n1 -r1

waveform.amplitude_probability_distribution(
    iq, analysis_bandwidth_Hz=analysis_bandwidth_Hz, **apd_bins
).to_xarray()

In [ ]:
fig, ax = plt.subplots()
apd.plot(ax=ax)

ax.set_yscale('gamma-qq', k=1, db_ordinal=True)
figures.label_axis(ax.xaxis, apd, 'channel_power', tick_units=False)
figures.label_axis(ax.yaxis, apd)

### Putting it all together

In [ ]:
# %%timeit -n1 -r1

from edge_sensor import dataset

del iq

iq = simulated_awgn(
    duration, sample_rate_Hz, power=sample_rate_Hz / analysis_bandwidth_Hz, xp=cp, pinned_cuda=True
)

analysis_spec = {
    'power_time_series': {'detector_period': 10e-3, 'detectors': ('rms', 'peak')},
    'cyclic_channel_power': {
        'cyclic_period': 10e-3,
        'detector_period': 1e-3 / 15 / 4,
        'detectors': ('rms', 'peak'),
        'cyclic_statistics': ('min', 'mean', 'max'),
    },
    'persistence_spectrum': {
        # 'window': 'flattop',
        'window': 'hamming',  # 'hamming', 'blackman', or 'blackmanharris'
        'resolution': 15e3,
        'quantiles': [0.5, 0.75, 0.95, 0.99, 1],
        'truncate': True
    },
    'amplitude_probability_distribution': {
        'power_low': -40,
        'power_high': 15,
        'power_count': 221,  # 0.25 dB resolution
    },
}

filter_spec = {
    'fft_size': 1024,
    'window': 'hamming',  # 'hamming', 'blackman', or 'blackmanharris'
}

dataset.from_spec(
    iq,
    sample_rate_Hz,
    analysis_bandwidth_Hz=analysis_bandwidth_Hz,
    filter_spec=filter_spec,
    analysis_spec=analysis_spec,
)

In [ ]:
def profile():
    return waveform.from_spec(
        iq,
        sample_rate_Hz,
        analysis_bandwidth_Hz=analysis_bandwidth_Hz,
        filter_spec=filter_spec,
        analysis_spec=analysis_spec,
    )

benchmark(profile, n_repeat=2)